In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.fft import fft, fftfreq

In [ ]:
#directories
# we change the expected frequencies and directories based on the defect type
base_path = r"C:\Users\lenovo\thesis\classification\csv"
output_path=  r"C:\Users\lenovo\thesis\classification\polars"

input_files = [
    
    output_path+"\\IR007_1_110\\X110_DE_time.csv",
    output_path+"\\IR014_1_175\\IR014_1_175X175_DE_time.csv",
    output_path+"\\IR021_1_214\\IR021_1_214X214_DE_time.csv",
    
]

output_directories = [

   output_path+"\\irpolar7",
   output_path+"\\irpolar14",
   output_path+"\\irpolar21",
]
expected_freqs = [5.4152]


In [ ]:
#Calculate frequncy and amplitute as key components of polar
#https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher
def calculate_polar_spectrum(data, fs, expected_freq):
    n = len(data)
    fft_result = fft(data)
    freqs = fftfreq(n, 1/fs)[:n//2]
    amplitude = 2.0 / n * np.abs(fft_result[0:n//2])
    return freqs, amplitude

In [ ]:
#Polar creation based on expected frequency of the faults
#https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher

def process_chunk(chunk_data, expected_freq, output_path):
    
    fs = 48000
    freqs, amplitude = calculate_polar_spectrum(chunk_data, fs, expected_freq)
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, polar=True)

    colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white', 'purple', 'orange']

    for i in range(len(amplitude)):
        if i < len(amplitude) // 2:
            rvalues = np.full(500, amplitude[i])
            npoints = 500
        else:
            rvalues = np.full(500, amplitude[i] * 10) 
            npoints = 500

        theta = np.linspace(0, 2 * np.pi, npoints) + (i * np.pi / len(amplitude))  

        ax.plot(theta, rvalues, color=colors[i % len(colors)], linewidth=1.5) 
        if freqs[i] <= expected_freq:
            expected_theta = 2 * np.pi * (freqs[i] / expected_freq)  
            ax.plot([expected_theta, expected_theta], [0, amplitude[i] * 10], color='gray', linestyle='--', linewidth=1.5)

    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_rticks([])  
    ax.set_yticklabels([])  
    ax.spines['polar'].set_visible(False)
    ax.set_title("Polar Spectrum")

    plt.tight_layout() 

    plt.savefig(output_path, dpi=300)
    plt.close(fig)  



In [ ]:
#split the signals to samples with 4800 datapoints with 50% overlap
def process_dataset(input_file, expected_freq, output_directory, chunk_size=4800, overlap=0.5):
    fs = 48000

    dataset = pd.read_csv(input_file, header=None)
    data = dataset.iloc[:, 0].values 

    nsamples = len(data)
    overlaps = int(chunk_size * overlap)
    nchunks = (nsamples - chunk_size) // overlaps + 1

    os.makedirs(output_directory, exist_ok=True)

    for j in range(nchunks):
        startid = j * overlaps
        endid = startid + chunk_size
        chunk_data = data[startid:endid]

        file_name = f"chunk{j + 1}_polar_spectrum.png"
        output_path = os.path.join(output_directory, file_name)

        # create polars
        process_chunk(chunk_data, expected_freq, output_path)

In [ ]:
for input_file, expected_freq, output_directory in zip(input_files, expected_freqs, output_directories):
    process_dataset(input_file, expected_freq, output_directory, chunk_size=4800, overlap=0.5)
